In [25]:
import mediapipe as mp
import cv2
import numpy as np
import time
import random

mphands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils


In [26]:
def begin(video):
    while True:
        ret, img = video.read()
        if not ret:
            continue

        img = cv2.flip(img, 1)
        frame = cv2.resize(img, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_LINEAR)
        height, width = frame.shape[:2]

        blank = np.zeros((height, width, 3), dtype=np.uint8)
        cv2.putText(blank, "ODD or EVEN", (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
        cv2.putText(blank, "Press Q to start...", (100, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.imshow("Hands", blank)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyWindow('Hands')
            return True

# Thumbs up detection screen with instruction + live video feed
def intro(video, lines):
    with mphands.Hands(max_num_hands=1, min_detection_confidence=0.7) as hands:
        while True:
            ret, img = video.read()
            if not ret:
                continue

            img = cv2.flip(img, 1)
            frame = cv2.resize(img, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_LINEAR)
            img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(img_rgb)

            # Display instruction text
            y = 50
            for i, line in enumerate(lines):
                cv2.putText(frame, line, (40, y + i * 50), cv2.FONT_HERSHEY_COMPLEX, 1.1, (0, 255, 0), 2)

            # Detect thumbs up
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    landmarks = [(lm.x, lm.y) for lm in hand_landmarks.landmark]

                    thumb_up = landmarks[4][1] < landmarks[3][1]  # Thumb tip above joint
                    fingers_down = all(landmarks[i][1] > landmarks[i-2][1] for i in [8, 12, 16, 20])

                    if thumb_up and fingers_down:
                        cv2.putText(frame, "Thumbs Up Detected!", (70, y + len(lines) * 50 + 30),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                        cv2.imshow("intro", frame)
                        cv2.waitKey(1500)
                        cv2.destroyWindow("intro")
                        return True

            cv2.imshow("intro", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                cv2.destroyWindow("intro")
                return False



In [27]:
def show_text_with_timer(video, display_text, seconds=5):
    import textwrap

    # Capture one frame to get camera dimensions
    ret, img = video.read()
    if not ret:
        print("❌ Camera read failed.")
        return False

    frame = cv2.resize(img, None, fx=1.5, fy=1.5)
    height, width = frame.shape[:2]

    start_time = time.time()

    # Ensure display_text is a list
    if isinstance(display_text, str):
        display_text = [display_text]

    # Wrap long lines (optional: change width as needed)
    wrapped_lines = []
    for line in display_text:
        wrapped_lines.extend(textwrap.wrap(line, width=30))

    while True:
        screen = np.zeros((height, width, 3), dtype=np.uint8)
        screen[:] = (30, 30, 30)  # dark background

        elapsed = time.time() - start_time
        remaining = int(seconds - elapsed)

        # Draw each line
        y_start = 100
        for i, line in enumerate(wrapped_lines):
            text_size = cv2.getTextSize(line, cv2.FONT_HERSHEY_SIMPLEX, 1.5, 3)[0]
            x = (width - text_size[0]) // 2  # Centered text
            y = y_start + i * 60
            cv2.putText(screen, line, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 3)

        # Show countdown timer
        if remaining >= 0:
            timer_text = f"Starting in: {remaining}"
            text_size = cv2.getTextSize(timer_text, cv2.FONT_HERSHEY_SIMPLEX, 1.2, 3)[0]
            x = (width - text_size[0]) // 2
            y = y_start + len(wrapped_lines) * 60 + 30
            cv2.putText(screen, timer_text, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 255), 3)
        else:
            break

        cv2.imshow("Blank Screen", screen)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyWindow("Blank Screen")
            return False

    cv2.destroyWindow("Blank Screen")
    return True


In [28]:
def thumbs_up(video):
    
    while True:
        data = mphands.Hands()
        suc,img = video.read()
        frame = cv2.resize(img, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_LINEAR)
        img1 = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        result = data.process(img1)
        tip_id = [4, 8, 12, 16, 20]
        lst = []
        thumb = 0
        if result.multi_hand_landmarks:
            for x in result.multi_hand_landmarks:
                for id,lm in enumerate(x.landmark):
                    cx = lm.x 
                    cy = lm.y 
                    lst.append([id,cx,cy])
                if len(lst)!=0 and len(lst)==21:
                    thumbs_up = lst[4][2] < lst[3][2]     # thumb tip above the thumb joint        
                    other_finger_down = all(lst[tip_id[i]][1] > lst[tip_id[i]-1][2] for i in range(1,5))
                    if thumbs_up and other_finger_down:
                        cv2.putText(frame, "Thumbs Up Detected!", (50, 50), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                        data.close()
                        cv2.imshow("Live Camera Feed", frame)
                        cv2.waitKey(1000)  # Show the message for 1 second
                        video.release()
                        cv2.destroyWindow("Live Camera Feed")
                        return True              
    

In [29]:
# # Thumbs up detection screen with instruction + live video feed
# def intro(video, lines):
#     with mphands.Hands(max_num_hands=1, min_detection_confidence=0.7) as hands:
#         while True:
#             ret, img = video.read()
#             if not ret:
#                 continue

#             img = cv2.flip(img, 1)
#             frame = cv2.resize(img, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_LINEAR)
#             img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#             results = hands.process(img_rgb)

#             # Display instruction text
#             y = 50
#             for i, line in enumerate(lines):
#                 cv2.putText(frame, line, (40, y + i * 50), cv2.FONT_HERSHEY_COMPLEX, 1.1, (0, 255, 0), 2)

#             # Detect thumbs up
#             if results.multi_hand_landmarks:
#                 for hand_landmarks in results.multi_hand_landmarks:
#                     landmarks = [(lm.x, lm.y) for lm in hand_landmarks.landmark]

#                     thumb_up = landmarks[4][1] < landmarks[3][1]  # Thumb tip above joint
#                     fingers_down = all(landmarks[i][1] > landmarks[i-2][1] for i in [8, 12, 16, 20])

#                     if thumb_up and fingers_down:
#                         cv2.putText(frame, "Thumbs Up Detected!", (70, y + len(lines) * 50 + 30),
#                                     cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
#                         cv2.imshow("intro", frame)
#                         cv2.waitKey(1500)
#                         cv2.destroyWindow("intro")
#                         return True

#             cv2.imshow("intro", frame)
#             if cv2.waitKey(1) & 0xFF == ord('q'):
#                 cv2.destroyWindow("intro")
#                 return False

In [30]:
def choose_odd_even(video, lines):
    mphands = mp.solutions.hands
    mp_drawing = mp.solutions.drawing_utils

    # Initialize MediaPipe hands
    hands = mphands.Hands(max_num_hands=1, min_detection_confidence=0.7)
    detection_start_time = None
    required_time = 5  # seconds
    tip_id = [4, 8, 12, 16, 20]

    while True:
        ret, frame = video.read()
        if not ret:
            continue

        frame = cv2.flip(frame, 1)
        frame = cv2.resize(frame, None, fx=1.5, fy=1.5)

        # Draw instructions
        y = 50
        for i, line in enumerate(lines):
            cv2.putText(frame, line, (40, y + i * 50), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (255, 255, 0), 2)
        cv2.putText(frame, "Hold gesture for 5 seconds", (40, 350), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(img_rgb)

        current_count = 0
        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                landmarks = []
                for id, lm in enumerate(hand_landmarks.landmark):
                    landmarks.append([id, lm.x, lm.y])

                if len(landmarks) == 21:
                    finger_count = []

                    # Index to Pinky (ignore thumb)
                    for i in range(1, 5):
                        if landmarks[tip_id[i]][2] < landmarks[tip_id[i]-2][2]:
                            finger_count.append(1)

                    count = sum(finger_count)
                    current_count = count if count in [2, 3] else 0

                    # Display count and label
                    if current_count > 0:
                        cv2.putText(frame, f"Detected: {current_count}", (50, 420), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 0, 0), 3)
                        label = "EVEN" if current_count == 2 else "ODD"
                        cv2.putText(frame, label, (50, 200), cv2.FONT_HERSHEY_COMPLEX, 3, (0, 255, 0), 3)

                        # Start or continue hold timer
                        if detection_start_time is None:
                            detection_start_time = time.time()
                        else:
                            elapsed = time.time() - detection_start_time
                            cv2.putText(frame, f"Holding: {int(required_time - elapsed)}s", (50, 300),
                                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
                            if elapsed >= required_time:
                                hands.close()
                                cv2.destroyWindow("odd_or_even")
                                return label
                    else:
                        detection_start_time = None
        else:
            detection_start_time = None

        # Show the frame
        cv2.imshow("odd_or_even", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    hands.close()
    cv2.destroyWindow("odd_or_even")
    return None

In [31]:
def Toss(video):
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
    
    required_hold_time = 3  # Hold your gesture for 3 seconds
    show_opponent_delay = 2  # Delay before showing opponent number

    tip_id = [4, 8, 12, 16, 20]
    current_count = None
    detection_start_time = None
    random_num = None
    show_opponent_time = None

    while True:
        ret, img = video.read()
        if not ret:
            continue

        frame = cv2.flip(img, 1)
        frame = cv2.resize(frame, None, fx=1.5, fy=1.5)
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        result = hands.process(img_rgb)
        lst = []
        finger_count = 0

        if result.multi_hand_landmarks:
            for hand in result.multi_hand_landmarks:
                for id, lm in enumerate(hand.landmark):
                    lst.append([id, lm.x, lm.y])

                if len(lst) == 21:
                    fingers = []

                    for i in range(1, 5):  # Index to pinky
                        if lst[tip_id[i]][2] < lst[tip_id[i] - 2][2]:
                            fingers.append(1)

                    # Thumb logic
                    if lst[4][0] < lst[3][0]:
                        fingers.append(1)

                    # Optional thumbs-up logic
                    thumbs_up = lst[4][2] < lst[3][2]
                    other_down = all(lst[tip_id[i]][2] > lst[tip_id[i] - 2][2] for i in range(1, 5))
                    if thumbs_up and other_down:
                        fingers = [1] * 6

                    finger_count = sum(fingers)

                    # Begin hold time check
                    if finger_count == current_count:
                        if detection_start_time is None:
                            detection_start_time = time.time()
                        elapsed = time.time() - detection_start_time
                        cv2.putText(frame, f"Holding: {int(elapsed)}s", (50, 450),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                        if elapsed >= required_hold_time:
                            # Generate opponent number once after hold
                            if random_num is None:
                                random_num = random.randint(0, 6)
                                show_opponent_time = time.time()  # start delay timer
                    else:
                        current_count = finger_count
                        detection_start_time = time.time()

        # If random_num is generated, wait 2 seconds then show it
        if random_num is not None:
            delay_elapsed = time.time() - show_opponent_time
            if delay_elapsed >= show_opponent_delay:
                cv2.putText(frame, f"Opponent: {random_num}", (30, 330),
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 3)
                cv2.putText(frame, f"You: {finger_count}", (30, 400),
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 0), 3)
                print(finger_count,random_num)
                # End after showing both
                cv2.imshow("Hands", frame)
                cv2.waitKey(2000)  # show for 2 more seconds
                hands.close()
                cv2.destroyWindow("Hands")
                return finger_count+random_num

        else:
            # Ongoing message
            cv2.putText(frame, f"Hold gesture for {required_hold_time} seconds", (30, 90),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)

        cv2.imshow("Hands", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    hands.close()
    cv2.destroyWindow("Hands")
    return None, None
    

In [32]:
def batting_or_bowl(video):
    # Initialize MediaPipe
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.5)
    mp_draw = mp.solutions.drawing_utils

    # Define boxes
    batting_box = ((170, 190), (270, 290))
    bowling_box = ((370, 190), (470, 290))

    # Hold time tracking
    batting_start, bowling_start = None, None
    required_time = 4  # seconds

    def draw_centered_text(frame, text, box, color):
        font = cv2.FONT_HERSHEY_SIMPLEX
        scale = 0.6
        thick = 2
        (tw, th), _ = cv2.getTextSize(text, font, scale, thick)
        x = box[0][0] + ((box[1][0] - box[0][0]) - tw) // 2
        y = box[0][1] + ((box[1][1] - box[0][1]) + th) // 2
        cv2.putText(frame, text, (x, y), font, scale, color, thick)
    while True:
        ret, frame = video.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        frame = cv2.resize(frame, None, fx=1.5, fy=1.5)
        h, w, _ = frame.shape
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(rgb)    

        # Draw boxes and text
        cv2.rectangle(frame, batting_box[0], batting_box[1], (0, 255, 0), 2)
        cv2.rectangle(frame, bowling_box[0], bowling_box[1], (0, 0, 255), 2)
        draw_centered_text(frame, "Batting", batting_box, (0, 255, 0))
        draw_centered_text(frame, "Bowling", bowling_box, (0, 0, 255))

        selection = None
        
        if result.multi_hand_landmarks:
            for lm in result.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame,lm,mphands.HAND_CONNECTIONS)
                
                # Point For Circle
                x = int((lm.landmark[8].x * w)) # X axis of point finger
                y = int((lm.landmark[8].y * h)) # Y axis of point Finger
                cv2.circle(frame,(x,y),10,(255,255,0),-1)
                
                # Batting box detection
                if batting_box[0][0] <= x <= batting_box[1][0] and batting_box[0][1] <= y <= batting_box[1][1]:
                    if batting_start is None:
                        batting_start = time.time()
                    elapsed = time.time() - batting_start
                    cv2.putText(frame, f"Holding: {int(elapsed)}s", (50, 430), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                    if elapsed >= required_time:
                        selection = "Batting"
                        break
                else:
                    batting_start = None

                # Bowling box detection
                if bowling_box[0][0] <= x <= bowling_box[1][0] and bowling_box[0][1] <= y <= bowling_box[1][1]:
                    if bowling_start is None:
                        bowling_start = time.time()
                    elapsed = time.time() - bowling_start
                    cv2.putText(frame, f"Holding: {int(elapsed)}s", (50, 430), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
                    if elapsed >= required_time:
                        selection = "Bowling"
                        break
                else:
                    bowling_start = None

        cv2.imshow("Batting or Bowling?", frame)
        if cv2.waitKey(1) & 0XFF == ord('q'):
            break
        if selection:
            print("✅ Selected:", selection)
            time.sleep(1)
            break
    # Clean up if user quits
    hands.close()
    cv2.destroyWindow("Batting or Bowling?")
    return selection  # or some default value       

In [33]:


def play(video,who,target):
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
    tip_ids = [4, 8, 12, 16, 20]

    desired_width, desired_height = 960, 720

    values = [0]
    total = 0
    storing = False
    black_screen_start = None
    system_number = None
    confirmed_number = None
    timer_start = None
    result_message = None

    while True:
        ret, frame = video.read()
        if not ret:
            continue

        frame = cv2.flip(frame, 1)
        frame = cv2.resize(frame, (desired_width, desired_height))
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(img_rgb)

        # Game info
        role_text = f"Batting: {'User' if who == 'user' else 'System'} | Bowling: {'System' if who == 'user' else 'User'}"
        cv2.putText(frame, role_text, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 0), 2)
        cv2.putText(frame, f"Total: {total}", (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)
        cv2.putText(frame, f"Balls: {len(values)}", (20, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 255), 2)
        if target is not None:
            cv2.putText(frame, f"Target: {target}", (20, 140), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 255), 2)

        # Show black screen if storing or result
        if storing:
            black_elapsed = time.time() - black_screen_start
            black_frame = np.zeros((desired_height, desired_width, 3), dtype=np.uint8)

            if result_message:
                cv2.putText(black_frame, result_message, (250, 330), cv2.FONT_HERSHEY_SIMPLEX, 1.8, (0, 255, 0), 4)
                cv2.putText(black_frame, f"Final Total: {total}", (300, 420), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 3)
                cv2.imshow("Gesture Capture", black_frame)
                cv2.waitKey(3000)
                break

            if black_elapsed < 2:
                cv2.putText(black_frame, "Gesture Stored!", (320, 330), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 3)
                cv2.imshow("Gesture Capture", black_frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                continue
            else:
                storing = False
                confirmed_number = None
                timer_start = None

        count = 0
        if result.multi_hand_landmarks:
            for handLms in result.multi_hand_landmarks:
                lm_list = [(lm.x, lm.y) for lm in handLms.landmark]

                if len(lm_list) == 21:
                    # Detect 6 for thumbs up
                    thumb_up = lm_list[4][1] < lm_list[3][1]
                    others_down = all(lm_list[tip_ids[i]][1] > lm_list[tip_ids[i] - 2][1] for i in range(1, 5))

                    if thumb_up and others_down:
                        count = 6
                    else:
                        fingers = []
                        fingers.append(1 if lm_list[4][0] < lm_list[3][0] else 0)
                        for i in range(1, 5):
                            fingers.append(1 if lm_list[tip_ids[i]][1] < lm_list[tip_ids[i] - 2][1] else 0)
                        count = sum(fingers)

                    cv2.putText(frame, f"Detected: {count}", (50, 600), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 0), 3)

                    if confirmed_number != count and count != 0:
                        confirmed_number = count
                        timer_start = time.time()
                    elif confirmed_number == count and timer_start:
                        elapsed = time.time() - timer_start
                        cv2.putText(frame, f"Hold for: {int(3 - elapsed)}s", (50, 550), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
                        if elapsed >= 3:
                            system_number = random.randint(1, 6)
                            print(f"🧍 You: {count}, 🤖 System: {system_number}")

                            if count == system_number:
                                result_message = "OUT !!!!"
                                storing = True
                                black_screen_start = time.time()
                                continue

                            values.append(count)
                            total += count if who == "user" else system_number

                            # Check if won
                            if target is not None and total >= target:
                                result_message = "🏆 WON!"
                                storing = True
                                black_screen_start = time.time()
                                continue

                            storing = True
                            black_screen_start = time.time()
                    else:
                        timer_start = None
                        confirmed_number = None
        else:
            timer_start = None
            confirmed_number = None

        if system_number:
            cv2.putText(frame, f"System: {system_number}", (700, 600), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)

        cv2.imshow("Gesture Capture", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # cap.release()
    hands.close()
    cv2.destroyAllWindows()
    print("✅ Final Total:", total)
    return total


In [34]:
# Main control function
def full():
    video = cv2.VideoCapture(2)  # Use 0 unless you know index 2 is correct
    if not video.isOpened():
        print("❌ Cannot open camera")
        return

    begin_value = begin(video)
    if begin_value:
        choose = intro(video, [
            "ODD or EVEN",
            "Choose on next Page",
            "(ODD = 3 fingers, EVEN = 2 fingers)",
            "Show thumbs up to continue"
        ])
        
        if choose:
            odd_even = choose_odd_even(video, ["(ODD = 3 fingers, EVEN = 2 fingers)"]) 
            if odd_even:
                print("✅ Result:", odd_even)
                messages = [
                f"You chose {odd_even}",
                "TOSS",
                "Show the number 1 to 6 ...",
                
                
    ]
                O_E = 0 if odd_even=="EVEN" else 1
                show_text_with_timer(video,messages)
                toss = Toss(video)
                print(toss)
                if (toss%2==O_E==0) or (toss%2==1==O_E):
                    show_text_with_timer(video,["Selection Upto You","Select Batting Or Bowling On Next..."])
                    bat_or_bowl = batting_or_bowl(video)
                    # print(bat_or_bowl)
                    show_text_with_timer(video,[f"You Chose chose {bat_or_bowl}"])
                    show_text_with_timer(video,["First Innings"])
                    if bat_or_bowl == "Batting":
                        first_innings = play(video,"user",None)
                        show_text_with_timer(video,[f"You Scored {first_innings}","Now Second Innings"])
                        second_innings = play(video,"system",first_innings)
                        if first_innings > second_innings:
                            show_text_with_timer(video,["You Won Fucker"])
                        else:
                            show_text_with_timer("You Loose Fucker !!!!")  
                    else:
                        first_innings = play(video,"system",None)
                        show_text_with_timer(video,[f"You Scored {first_innings}","Now Second Innings"])
                        second_innings = play(video,"user",first_innings)
                        if first_innings > second_innings:
                            show_text_with_timer(video,["You Won Fucker"])
                        else:
                            show_text_with_timer("You Loose Fucker !!!!")         
                else:
                    bat_or_bowl = random.choice(["Batting","Bowling"])
                    show_text_with_timer(video,['Selection upto Computer',f"Computer Chose  {bat_or_bowl}"])
                    show_text_with_timer(video,["First Innings"])
                    first_innings = play(video,"system",None)
                    show_text_with_timer(video,[f"System Scored {first_innings}",f"You Need to Score More than {first_innings} ","Now Second Innings"])
                    second_innings = play(video,"user",first_innings)
                    if first_innings > second_innings:
                        show_text_with_timer(video,["You Loose Fucker"])
                    else:
                        show_text_with_timer("You Won Fucker !!!!") 
                    
                    
        else:
            print("❌ Thumbs up not detected. Exiting...")
            

    else:
        print("❌ Begin screen skipped or failed.")

    video.release()
    cv2.destroyAllWindows()

In [35]:
full()

✅ Result: EVEN
4 5
9
🧍 You: 6, 🤖 System: 2
🧍 You: 5, 🤖 System: 3
🧍 You: 2, 🤖 System: 1
🧍 You: 1, 🤖 System: 6
🧍 You: 6, 🤖 System: 2
🧍 You: 5, 🤖 System: 2
🧍 You: 2, 🤖 System: 1
🧍 You: 3, 🤖 System: 2
🧍 You: 2, 🤖 System: 4
🧍 You: 5, 🤖 System: 6
🧍 You: 6, 🤖 System: 3
🧍 You: 1, 🤖 System: 6
🧍 You: 3, 🤖 System: 4
🧍 You: 2, 🤖 System: 1
🧍 You: 5, 🤖 System: 5
✅ Final Total: 43
🧍 You: 2, 🤖 System: 2
✅ Final Total: 0
